<a href="https://colab.research.google.com/github/GB1111/Coursera_Capstone/blob/master/Pune_Chinese_Restaurants_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis


In [2]:
path = 'https://raw.githubusercontent.com/GB1111/Coursera_Capstone/master/pune_data.csv'
pune_data = pd.read_csv(path)
pune_data.head()

,Postal Code,Place Name,County,Community,Latitude,Longitude,Accuracy
0,410301,Khandala,Pune,Maval,18.7589,73.3694,4
1,410302,R P T S Khandala,Pune,Maval,18.7589,73.3694,1
2,410401,Lonavala,Pune,Maval,18.7528,73.4057,4
3,410401,Kusgaon BK,Pune,Maval,18.7528,73.4057,3
4,410401,Lonavala Bazar,Pune,Maval,18.7528,73.4057,3


In [3]:
pune_data = pune_data[['Postal Code', 'Community', 'Place Name', 'Latitude', 'Longitude']]
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [5]:
pune_data.head()

,Postal Code,Community,Place Name,Latitude,Longitude
0,410301,Maval,Khandala,18.7589,73.3694
1,410302,Maval,R P T S Khandala,18.7589,73.3694
2,410401,Maval,Lonavala,18.7528,73.4057
3,410401,Maval,Kusgaon BK,18.7528,73.4057
4,410401,Maval,Lonavala Bazar,18.7528,73.4057


In [4]:
print('The dataframe has {} communities and {} places.'.format(
        len(pune_data['Community'].unique()),
        pune_data.shape[0]
    )
)

The dataframe has 20 communities and 791 places.


In [6]:
pune_data.groupby('Community').count()

,Postal Code,Place Name,Latitude,Longitude
Community,,,,
Ambegaon,43,43,43,43
Baramati,56,56,56,56
Bhor,69,69,69,69
Daund,52,52,52,52
Haveli,76,76,76,76
Indapur,50,50,50,50
Junnar,68,68,68,68
Khed,68,68,68,68
Maval,38,38,38,38


In [7]:
CLIENT_ID = 'L1M31TY1GR4DEX4NKN0BDORMY3VI1UFT2A5NDINLYKX15K1F' # your Foursquare ID
CLIENT_SECRET = 'MPTNBTGWLIKLHBORPIHLQNBQRM21LJTE1QTSP1KJ1BF0VY2U' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: L1M31TY1GR4DEX4NKN0BDORMY3VI1UFT2A5NDINLYKX15K1F
CLIENT_SECRET:MPTNBTGWLIKLHBORPIHLQNBQRM21LJTE1QTSP1KJ1BF0VY2U


In [8]:
pune_data.loc[0, 'Place Name']

'Khandala'

In [9]:
place_latitude = pune_data.loc[0, 'Latitude'] # neighborhood latitude value
place_longitude = pune_data.loc[0, 'Longitude'] # neighborhood longitude value

place_name = pune_data.loc[0, 'Place Name'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(place_name, 
                                                               place_latitude, 
                                                               place_longitude))

Latitude and longitude values of Khandala are 18.7589, 73.3694.


In [10]:
LIMIT = 100
radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    place_latitude, 
    place_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=L1M31TY1GR4DEX4NKN0BDORMY3VI1UFT2A5NDINLYKX15K1F&client_secret=MPTNBTGWLIKLHBORPIHLQNBQRM21LJTE1QTSP1KJ1BF0VY2U&v=20180605&ll=18.7589,73.3694&radius=500&limit=100'

In [0]:
import requests

In [0]:
results = requests.get(url).json()

In [0]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,High Point @ Dukes Retreat,Hotel Bar,18.761639,73.370665
1,Dukes Retreat Khandala,Hotel,18.761514,73.370721
2,Shooting point,Outdoors & Recreation,18.758637,73.373100
3,Velvett Country,Resort,18.755733,73.367764


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
pune_venues = getNearbyVenues(names=pune_data['Place Name'],
                                   latitudes=pune_data['Latitude'],
                                   longitudes=pune_data['Longitude']
                                 )
print(pune_venues.shape)

Khandala
R P T S Khandala
Lonavala
Kusgaon BK
Lonavala Bazar
Ambavane
Ins Shivaji Lonavale
Kurwande
Kaivalyadham
Karanjgaon
Kamshet
Karunj
Malawali
Karla
Takave KH
Vehergaon
Nane
Pawananagar
Shivali
Bhose
Chakan
Mahalunge Ingale
Vasuli
Waki BK
Ambethan
Vadgaon Ghenanad
Khalumbre
Shive
Rase
Pimpalgaon Tarfe Chakan
Kuruli
Koyali
Nighoje
Kalus
Gadad
Bahul
Rajur
Parunde
Junnar
Tambe
Devale
Inglun
Khanapur
Vadaj
Anjanavale
Kusur
Khangaon
Amboli
Aptale
Yenere
Khamgaon
Khadkumbe(Exptl)
Ballalwadi
Manikdoh
Chavand
Nimgiri
Godre
Nirgude
Utchil
Manchar
Landewadi
Chandoli BK
Vadgaon Kashimbeg
Pimpalgaon Mahalunge
Narodi
Ozar
Khodad
Yedgaon
Pimparkhed
Pargaon Tarfe Ala
Ranjani
Valati
Narayangaon
Nimgaon Sava
Khamundi
Pimpri Pendhar
Jambhut
Hivare BK
Amboli
Vaphgaon
Pimpri BK
Rajgurunagar
Palu
Donda
Nimgaon Khed
Chinchoshi
Pait
Yeniye BK
Kurkundi
Varude
Kude BK
Sandbhorwadi(Exptl)
Kiwale
Kanhersar
Retwadi
Shiroli
Gulani
Ghotawadi
Dawadi
Talegaon Dabhade
Dhamane
Urse
Chandkhed
Adhale BK
Bebedhol
Ord

In [19]:
pune_venues.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar
1,Khandala,18.7589,73.3694,Dukes Retreat Khandala,18.761514,73.370721,Hotel
2,Khandala,18.7589,73.3694,Shooting point,18.758637,73.373100,Outdoors & Recreation
3,Khandala,18.7589,73.3694,Velvett Country,18.755733,73.367764,Resort
4,R P T S Khandala,18.7589,73.3694,High Point @ Dukes Retreat,18.761639,73.370665,Hotel Bar


In [20]:
pune_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
9 DRD,32,32,32,32,32,32
A.R. Shala,15,15,15,15,15,15
Adhale BK,4,4,4,4,4,4
Airport (Pune),4,4,4,4,4,4
Akurdi,4,4,4,4,4,4
...,...,...,...,...,...,...
Waki,2,2,2,2,2,2
Warje,5,5,5,5,5,5
Yavat,1,1,1,1,1,1


In [21]:
print('There are {} uniques categories.'.format(len(pune_venues['Venue Category'].unique())))

There are 85 uniques categories.


**Analyze Each Neighborhood**

In [22]:
# one hot encoding
pune_onehot = pd.get_dummies(pune_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
pune_onehot['Neighbourhood'] = pune_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [pune_onehot.columns[-1]] + list(pune_onehot.columns[:-1])
pune_onehot = pune_onehot[fixed_columns]

pune_onehot.head()

,Neighbourhood,ATM,Antique Shop,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bed & Breakfast,Bistro,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gym,Gym / Fitness Center,Hotel,Hotel Bar,...,Juice Bar,Lake,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Nature Preserve,Nightclub,North Indian Restaurant,Outdoors & Recreation,Outlet Mall,Park,Pizza Place,Platform,Resort,Rest Area,Restaurant,River,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Stables,Stadium,Tea Room,Tennis Court,Theater,Track,Trail,Train Station,Vegetarian / Vegan Restaurant
0,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,R P T S Khandala,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
pune_onehot.shape

(877, 86)

In [24]:
pune_grouped = pune_onehot.groupby('Neighbourhood').mean().reset_index()
pune_grouped

,Neighbourhood,ATM,Antique Shop,Asian Restaurant,Auto Workshop,BBQ Joint,Bakery,Bed & Breakfast,Bistro,Breakfast Spot,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Convenience Store,Creperie,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store,Farm,Farmers Market,Fast Food Restaurant,Forest,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gym,Gym / Fitness Center,Hotel,Hotel Bar,...,Juice Bar,Lake,Lounge,Maharashtrian Restaurant,Market,Men's Store,Mobile Phone Shop,Motel,Motorcycle Shop,Movie Theater,Multicuisine Indian Restaurant,Nature Preserve,Nightclub,North Indian Restaurant,Outdoors & Recreation,Outlet Mall,Park,Pizza Place,Platform,Resort,Rest Area,Restaurant,River,Scenic Lookout,Seafood Restaurant,Shopping Mall,Snack Place,Soccer Field,South Indian Restaurant,Southern / Soul Food Restaurant,Spa,Stables,Stadium,Tea Room,Tennis Court,Theater,Track,Trail,Train Station,Vegetarian / Vegan Restaurant
0,9 DRD,0.0,0.0,0.0,0.0,0.0,0.03125,0.000000,0.000000,0.0,0.03125,0.0,0.00,0.0,0.062500,0.03125,0.062500,0.0,0.000000,0.09375,0.0,0.0,0.000000,0.0,0.0625,0.0,0.0,0.0,0.0,0.03125,0.0,0.03125,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.062500,0.0,...,0.03125,0.0,0.031250,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.03125,0.03125,0.0,0.0,0.0,0.062500,0.0,0.0,0.0,0.0,0.00,0.03125,0.03125,0.03125,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,A.R. Shala,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.066667,0.0,0.00000,0.0,0.00,0.0,0.066667,0.00000,0.066667,0.0,0.066667,0.00000,0.0,0.0,0.066667,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.066667,0.0,0.066667,0.066667,0.066667,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.066667,0.0,0.0,0.0,0.0,0.20,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.066667,0.0,0.0,0.0
2,Adhale BK,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.25,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.5,0.0
3,Airport (Pune),0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.25,0.00000,0.00000,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.00,0.25000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,Akurdi,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.25000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.25000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.00,0.00000,0.00000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.25,0.00000,0.00000,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,Waki,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.00000,0.0,0.00,0.0,0.000000,0.00000,0.000000,0.0,0.000000,0.00000,0.0,0.0,0.000000,0.0,0.0000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...

In [25]:
pune_grouped.shape

(176, 86)

In [26]:
chinese_grouped = pune_grouped[['Neighbourhood', 'Chinese Restaurant']]
chinese_grouped.head()

,Neighbourhood,Chinese Restaurant
0,9 DRD,0.062500
1,A.R. Shala,0.066667
2,Adhale BK,0.000000
3,Airport (Pune),0.000000
4,Akurdi,0.000000


**Cluster Neighborhoods**




In [27]:
# set number of clusters
kclusters = 3

chinese_grouped_clustering = chinese_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chinese_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [0]:
pune_merged = chinese_grouped.copy()

# add clustering labels
pune_merged["Cluster Labels"] = kmeans.labels_

In [29]:
pune_merged.head()

,Neighbourhood,Chinese Restaurant,Cluster Labels
0,9 DRD,0.062500,1
1,A.R. Shala,0.066667,1
2,Adhale BK,0.000000,0
3,Airport (Pune),0.000000,0
4,Akurdi,0.000000,0


In [0]:
pune_merged = pune_merged.join(pune_venues.set_index("Neighbourhood"), on="Neighbourhood")



In [31]:
print(pune_merged.shape)
pune_merged.head()

(877, 9)


,Neighbourhood,Chinese Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,9 DRD,0.0625,1,18.5685,73.9158,Natural Ice Cream,18.567612,73.917748,Ice Cream Shop
0,9 DRD,0.0625,1,18.5685,73.9158,Perks 'N' Brews,18.567873,73.914766,Coffee Shop
0,9 DRD,0.0625,1,18.5685,73.9158,Falahaar,18.565390,73.918158,Juice Bar
0,9 DRD,0.0625,1,18.5685,73.9158,Cafe Coffee Day,18.566079,73.918249,Coffee Shop
0,9 DRD,0.0625,1,18.5685,73.9158,Chopsticks Spice Malabar,18.565245,73.913116,Southern / Soul Food Restaurant


In [32]:
# sort the results by Cluster Labels
print(pune_merged.shape)
pune_merged.sort_values(["Cluster Labels"], inplace=True)
pune_merged

(877, 9)


,Neighbourhood,Chinese Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
81,Lonavala,0.0,0,18.7528,73.4057,Lonavala Railway Station,18.749728,73.408441,Train Station
99,Nana Peth,0.0,0,18.5372,73.8551,Suruchi Fast food,18.538394,73.850650,Snack Place
100,Narayan Peth,0.0,0,18.5333,73.8667,AISSMS Dual Football Ground,18.530317,73.865011,Soccer Field
100,Narayan Peth,0.0,0,18.5333,73.8667,The Millers,18.531992,73.870609,Restaurant
100,Narayan Peth,0.0,0,18.5333,73.8667,Baobab,18.532201,73.870830,Restaurant
...,...,...,...,...,...,...,...,...,...
11,Armament,0.1,2,18.5131,73.7825,FishLand,18.512353,73.782216,Seafood Restaurant
11,Armament,0.1,2,18.5131,73.7825,Vaishnavi Snacks,18.512297,73.782144,Snack Place
11,Armament,0.1,2,18.5131,73.7825,Peter's Momo Center,18.515041,73.782098,Chinese Restaurant
11,Armament,0.1,2,18.5131,73.7825,Chinjabi/Masemari,18.515565,73.781699,Restaurant


In [33]:
grouped = pune_merged.groupby('Cluster Labels')['Chinese Restaurant'].count()
grouped.head()

Cluster Labels
0    674
1    173
2     30
Name: Chinese Restaurant, dtype: int64

In [34]:
!pip install geocoder

In [0]:
import geocoder

In [36]:
address = 'Pune, IN'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Pune are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Pune are 18.521428, 73.8544541.


In [0]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pune_merged['Neighbourhood Latitude'], pune_merged['Neighbourhood Longitude'], pune_merged['Neighbourhood'], pune_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Examine Clusters** 

In [39]:
#Cluster 0
pune_merged.loc[pune_merged['Cluster Labels'] == 0]

,Neighbourhood,Chinese Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
81,Lonavala,0.0,0,18.7528,73.4057,Lonavala Railway Station,18.749728,73.408441,Train Station
99,Nana Peth,0.0,0,18.5372,73.8551,Suruchi Fast food,18.538394,73.850650,Snack Place
100,Narayan Peth,0.0,0,18.5333,73.8667,AISSMS Dual Football Ground,18.530317,73.865011,Soccer Field
100,Narayan Peth,0.0,0,18.5333,73.8667,The Millers,18.531992,73.870609,Restaurant
100,Narayan Peth,0.0,0,18.5333,73.8667,Baobab,18.532201,73.870830,Restaurant
...,...,...,...,...,...,...,...,...,...
49,Ghorpuri Bazar,0.0,0,18.5372,73.8551,cafeteria l&t pune,18.535149,73.851362,Café
49,Ghorpuri Bazar,0.0,0,18.5372,73.8551,Suruchi Fast food,18.538394,73.850650,Snack Place
48,Ganeshkhind,0.0,0,18.5333,73.8667,The Mills,18.532232,73.870892,Outlet Mall
49,Ghorpuri Bazar,0.0,0,18.5372,73.8551,Shopper's Stop,18.534332,73.851900,Department Store


In [40]:
#Cluster 1
pune_merged.loc[pune_merged['Cluster Labels'] == 1]

,Neighbourhood,Chinese Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,9 DRD,0.062500,1,18.5685,73.9158,Sweet Chariot,18.565050,73.916055,Bakery
0,9 DRD,0.062500,1,18.5685,73.9158,Shree Krishna,18.565101,73.917951,Indian Restaurant
162,Vadgaon Sheri,0.062500,1,18.5685,73.9158,Sujata Mastani,18.565400,73.915169,Dessert Shop
162,Vadgaon Sheri,0.062500,1,18.5685,73.9158,Madras Cafe,18.566958,73.914152,South Indian Restaurant
162,Vadgaon Sheri,0.062500,1,18.5685,73.9158,Fun Fitness Football Ground,18.567465,73.915817,Soccer Field
...,...,...,...,...,...,...,...,...,...
45,Film Institute,0.066667,1,18.5148,73.8358,Reshma Bhurji,18.511800,73.837429,Snack Place
45,Film Institute,0.066667,1,18.5148,73.8358,Kamala Nehru Park,18.517050,73.834099,Garden
45,Film Institute,0.066667,1,18.5148,73.8358,Cafe Paradise,18.512869,73.839882,Café
45,Film Institute,0.066667,1,18.5148,73.8358,PYC Club,18.517336,73.839427,Gym


In [41]:
#Cluster 2
pune_merged.loc[pune_merged['Cluster Labels'] == 2]

,Neighbourhood,Chinese Restaurant,Cluster Labels,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
151,Sus,0.1,2,18.5131,73.7825,Vaishnavi Snacks,18.512297,73.782144,Snack Place
151,Sus,0.1,2,18.5131,73.7825,FishLand,18.512353,73.782216,Seafood Restaurant
151,Sus,0.1,2,18.5131,73.7825,"Cafe Peter Donuts, Bavdhan",18.510418,73.781944,Café
151,Sus,0.1,2,18.5131,73.7825,Domino's Pizza,18.517000,73.782000,Pizza Place
16,Bavdhan,0.1,2,18.5131,73.7825,Cafe PeterDonuts,18.510525,73.782382,Café
151,Sus,0.1,2,18.5131,73.7825,Cafe PeterDonuts,18.510525,73.782382,Café
151,Sus,0.1,2,18.5131,73.7825,Chinjabi/Masemari,18.515565,73.781699,Restaurant
151,Sus,0.1,2,18.5131,73.7825,Peter's Momo Center,18.515041,73.782098,Chinese Restaurant
151,Sus,0.1,2,18.5131,73.7825,Cafe Durga,18.512556,73.782179,Café
151,Sus,0.1,2,18.5131,73.7825,Eatsome,18.515559,73.781469,Fast Food Restaurant


**Observations**

Most of Chinese restaurants are in Cluster 2 which is around Sus, Bavdhan and Armament areas and lowest (close to zero) in Cluster 0 areas which are Lonavala, Narayan Peth, Yerwada areas. Also, there are good opportunities to open near Vadgaon Sheri, Film Institute as the competition seems to be low. Looking at nearby venues, it seems Cluster 1 might be a good location as there are not a lot of Chinese restaurants in these areas. Therefore, this project recommends the entrepreneur to open a Chinese restaurant in these locations with little to no competition.